In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import sys
import os
import pickle
import torch
from IPython.display import HTML




# Allow imports from the parent directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from main import tools

In [7]:
# File paths for preprocessed data and fitted data
pre_file_path = 'datas/datas_pre.pkl'

with open(pre_file_path, 'rb') as f:
    datas_pre = pickle.load(f)


In [8]:
data = datas_pre[0]

In [12]:
data.Z

tensor([[-0.3333, -0.3892, -0.3994,  ...,  0.4142,  0.2598,  0.1047],
        [-0.3742, -0.3534, -0.3221,  ...,  0.1521,  0.0804,  0.0858],
        [-0.1824, -0.1479, -0.1287,  ...,  0.2326,  0.3055,  0.3890],
        [-0.0474, -0.0514, -0.0452,  ...,  0.4877,  0.5154,  0.5331],
        [ 0.0283,  0.0397,  0.0416,  ...,  0.5270,  0.5412,  0.5317]])

In [9]:
len(datas_pre), data.m

(1, 2500)

In [13]:
f = data.plot_observations(frame=3, gif = True, full = False)
HTML(f.to_html5_video())


In [3]:
fit_file_path = 'datas/datas_fit.pkl'

with open(fit_file_path, 'rb') as f:


    datas_fit = pickle.load(f)

In [4]:
data = datas_fit[0]

In [7]:
indices = torch.randperm(data.m)[:2500]
f = data.plot_vel(indices, frame=0, color="blue", gif = True)
HTML(f.to_html5_video())

In [5]:
vel = ode.Vel_hat(data, scale=False)

NameError: name 'ode' is not defined

In [11]:
f  = lambda txy: vel(txy[0], torch.atleast_2d(txy[1:])  )[0] 
t = data.T[3]
XY = data.XY
L1 = 0.1
L2 = 0.75
tXY = torch.cat([t.repeat(XY.shape[0],1), 
                        XY], dim = -1)

Jacobians = torch.vmap(torch.func.jacrev(f))(tXY)

col0 = Jacobians[:, :, 0]
col0_norms = torch.linalg.vector_norm(col0, ord=2, dim=1)
zero = torch.zeros(1, device=XY.device)
col0_norms = torch.max(zero, col0_norms - L1) #** 2

block2x2 = Jacobians[:, :, 1:]
block2x2_norms = torch.linalg.matrix_norm(block2x2, ord=2, dim=(1, 2))
block2x2_norms =  torch.max(zero, block2x2_norms - L2)#** 2

In [14]:
torch.sum(col0_norms == 0)

tensor(2566)